In [5]:
# Análise de Dados - E-commerce (Brasil)

Pipeline completo:
- Carregamento e preparação dos dados
- Feature engineering e tratamento de outliers
- EDA (gráficos + estatísticas)
- KPIs, sazonalidade e elasticidade
- Inferência estatística e exportação das bases

SyntaxError: invalid syntax (20603348.py, line 3)

In [1]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

ModuleNotFoundError: No module named 'matplotlib'

## 0. Setup & caminhos

In [ ]:
# Ajusta dinamicamente o diretório raiz para garantir caminhos relativos corretos
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR if NOTEBOOK_DIR.name != "notebooks" else NOTEBOOK_DIR.parent
os.chdir(PROJECT_ROOT)
print("Diretório atual:", PROJECT_ROOT)

pd.set_option("display.max_columns", 50)
plt.style.use("default")

os.makedirs("kpis", exist_ok=True)
os.makedirs("images", exist_ok=True)

## 1. Carregamento dos dados

In [ ]:
data_dir = PROJECT_ROOT / "data"

path_dim_customer = data_dir / "DIM_Customer.csv"
path_dim_products = data_dir / "DIM_Products.csv"
path_dim_delivery = data_dir / "DIM_Delivery.csv"
path_fact_orders = data_dir / "FACT_Orders.csv"
path_dim_shopping = data_dir / "DIM_Shopping.csv"


dim_customer = pd.read_csv(path_dim_customer)
dim_products = pd.read_csv(path_dim_products)
dim_delivery = pd.read_csv(path_dim_delivery)
fact_orders = pd.read_csv(path_fact_orders)
dim_shopping = pd.read_csv(path_dim_shopping)

dim_customer.head()

## 2. Qualidade dos dados & tipos

In [ ]:
def resumo_qualidade(df, nome):
    print(f"\n===== {nome} =====")
    print("Shape:", df.shape)
    print("Tipos:\n", df.dtypes)
    print("\nNulos por coluna:\n", df.isna().sum())
    if "Id" in df.columns:
        print("Duplicados por Id:", df.duplicated(subset=["Id"]).sum())
    else:
        print("Duplicados por Id: N/A")


def trim_string_columns(df):
    obj_cols = df.select_dtypes(include=["object"]).columns
    for col in obj_cols:
        df[col] = df[col].astype(str).str.strip()
    return df


dim_customer = trim_string_columns(dim_customer)
dim_products = trim_string_columns(dim_products)
dim_delivery = trim_string_columns(dim_delivery)
fact_orders = trim_string_columns(fact_orders)
dim_shopping = trim_string_columns(dim_shopping)

for nome, df_tmp in [
    ("DIM_Customer", dim_customer),
    ("DIM_Products", dim_products),
    ("DIM_Delivery", dim_delivery),
    ("FACT_Orders", fact_orders),
    ("DIM_Shopping", dim_shopping),
]:
    resumo_qualidade(df_tmp, nome)

## 3. Conversão de tipos e joins

In [ ]:
fact_orders["Order_Date"] = pd.to_datetime(fact_orders["Order_Date"])
dim_delivery["D_Date"] = pd.to_datetime(dim_delivery["D_Date"])
dim_delivery["D_Forecast"] = pd.to_datetime(dim_delivery["D_Forecast"])

before = len(dim_delivery)
dim_delivery = dim_delivery.dropna(subset=["D_Date", "D_Forecast"])
print(f"Registros removidos em DIM_Delivery (sem datas críticas): {before - len(dim_delivery)}")

before = len(fact_orders)
fact_orders = fact_orders.dropna(subset=["Order_Date", "Total", "Subtotal"])
print(f"Registros removidos em FACT_Orders (dados críticos ausentes): {before - len(fact_orders)}")

before_dup = fact_orders.duplicated(subset=["Id"]).sum()
print(f"Pedidos duplicados em FACT_Orders: {before_dup}")
fact_orders = fact_orders.drop_duplicates(subset=["Id"])


df = (
    fact_orders
    .merge(dim_delivery, on="Id", how="left", suffixes=("", "_deliv"))
    .merge(dim_customer[["Id", "Customer_Id", "State", "Region"]], on="Id", how="left")
    .merge(dim_products[["Id", "Category", "Subcategory"]], on="Id", how="left")
)

print("Sem correspondência em DIM_Delivery:", df["D_Date"].isna().sum())
print("Sem correspondência em DIM_Customer:", df["Customer_Id"].isna().sum())
print("Sem correspondência em DIM_Products:", df["Category"].isna().sum())

df.head()

## 4. Feature Engineering

In [ ]:
df["delivery_delay_days"] = (df["D_Date"] - df["D_Forecast"]).dt.days
df["delivery_lead_time"] = (df["D_Date"] - df["Order_Date"]).dt.days
df["is_late"] = (df["D_Date"] > df["D_Forecast"]).astype(int)
df["is_confirmed"] = (df["Purchase_Status"] == "Confirmado").astype(int)
df["freight_share"] = df["P_Sevice"] / df["Total"]
df["discount_abs"] = df["Discount"] * df["Subtotal"]
df["month"] = df["Order_Date"].dt.to_period("M").astype(str)

df.head()

## 5. Tratamento de outliers

In [ ]:
def detectar_outliers_iqr(series, k=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    li = q1 - k * iqr
    ls = q3 + k * iqr
    mask = (series >= li) & (series <= ls)
    return mask, li, ls


metricas_outlier = ["Total", "delivery_lead_time", "delivery_delay_days", "Discount"]
mask_global = pd.Series(True, index=df.index, dtype=bool)

for col in metricas_outlier:
    serie = df[col].dropna()
    mask_col = pd.Series(True, index=df.index, dtype=bool)
    if not serie.empty:
        mask_values, li, ls = detectar_outliers_iqr(serie)
        mask_col.loc[serie.index] = mask_values.values
        removidos = (~mask_col).sum()
        print(f"Outliers (IQR) em {col}: {removidos} | limites [{li:.2f}, {ls:.2f}]")
        mask_global &= mask_col


df_clean = df[mask_global].copy()
df_clean.reset_index(drop=True, inplace=True)
print(f"Registros totais: {len(df)}, sem outliers em métricas-chave: {len(df_clean)}")

## 6. Estatísticas descritivas


In [ ]:
def resumo_metricas(base, nome):
    print(f"===== Estatísticas ({nome}) =====")
    print(f"Ticket médio: R$ {base['Total'].mean():,.2f}")
    print(f"Lead time médio: {base['delivery_lead_time'].mean():.2f} dias")
    print(f"Atraso médio: {base['delivery_delay_days'].mean():.2f} dias")
    print(f"Proporção atrasos: {base['is_late'].mean():.2%}")
    cancel_rate = (base['Purchase_Status'] == 'Cancelado').mean()
    print(f"Cancelamentos: {cancel_rate:.2%}\n")


resumo_metricas(df, "original")
resumo_metricas(df_clean, "sem outliers")


## 7. EDA – Gráficos

In [ ]:
plt.figure()
plt.hist(df_clean["Total"], bins=50)
plt.title("Distribuição do Ticket (sem outliers)")
plt.tight_layout()
plt.savefig("images/hist_ticket.png", dpi=120)
plt.close()

plt.figure()
plt.hist(df_clean["delivery_lead_time"], bins=50)
plt.title("Lead Time (sem outliers)")
plt.tight_layout()
plt.savefig("images/hist_leadtime.png", dpi=120)
plt.close()

plt.figure()
plt.hist(df_clean["delivery_delay_days"], bins=50)
plt.title("Atraso na entrega (dias, sem outliers)")
plt.tight_layout()
plt.savefig("images/hist_delay.png", dpi=120)
plt.close()

plt.figure()
plt.hist(df_clean["Discount"], bins=50)
plt.title("Distribuição de descontos (sem outliers)")
plt.tight_layout()
plt.savefig("images/hist_discount.png", dpi=120)
plt.close()

plt.figure()
plt.boxplot(df_clean["Total"].dropna())
plt.title("Boxplot - Ticket")
plt.tight_layout()
plt.savefig("images/box_ticket.png", dpi=120)
plt.close()

plt.figure()
plt.boxplot(df_clean["delivery_lead_time"].dropna())
plt.title("Boxplot - Lead time")
plt.tight_layout()
plt.savefig("images/box_leadtime.png", dpi=120)
plt.close()

plt.figure()
plt.boxplot(df_clean["Discount"].dropna())
plt.title("Boxplot - Desconto")
plt.tight_layout()
plt.savefig("images/box_discount.png", dpi=120)
plt.close()

numeric_cols = [
    "Total",
    "Subtotal",
    "Discount",
    "discount_abs",
    "P_Sevice",
    "freight_share",
    "delivery_lead_time",
    "delivery_delay_days",
]

corr = df_clean[numeric_cols].corr()

plt.figure(figsize=(8, 6))
plt.imshow(corr, cmap="coolwarm", interpolation="nearest")
plt.colorbar()
plt.title("Matriz de correlação (sem outliers)")
plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=45)
plt.yticks(range(len(numeric_cols)), numeric_cols)
plt.tight_layout()
plt.savefig("images/correlacao.png", dpi=120)
plt.close()

## 8. KPIs, sazonalidade e elasticidade

In [ ]:
seasonality = (
    df_clean.groupby(["month", "Region"])
    .agg(
        revenue=("Total", "sum"),
        orders=("Id", "count"),
        late_rate=("is_late", "mean"),
    )
    .reset_index()
)

kpis_by_service = df_clean.groupby("Services").agg(
    orders=("Id", "count"),
    avg_ticket=("Total", "mean"),
    avg_lead_time=("delivery_lead_time", "mean"),
    avg_delay=("delivery_delay_days", "mean"),
    late_rate=("is_late", "mean"),
    cancel_rate=("Purchase_Status", lambda x: (x == "Cancelado").mean()),
    freight_share_mean=("freight_share", "mean"),
).reset_index()

kpis_by_payment = df_clean.groupby("payment").agg(
    orders=("Id", "count"),
    confirm_rate=("is_confirmed", "mean"),
    cancel_rate=("Purchase_Status", lambda x: (x == "Cancelado").mean()),
    avg_ticket=("Total", "mean"),
    avg_discount=("Discount", "mean"),
).reset_index()

kpis_by_region = df_clean.groupby("Region").agg(
    orders=("Id", "count"),
    avg_ticket=("Total", "mean"),
    avg_lead_time=("delivery_lead_time", "mean"),
    late_rate=("is_late", "mean"),
).reset_index()

kpis_by_category = (
    df_clean.groupby(["Category", "Subcategory"])
    .agg(
        orders=("Id", "count"),
        revenue=("Total", "sum"),
        avg_ticket=("Total", "mean"),
        avg_discount=("Discount", "mean"),
    )
    .reset_index()
)

bins = [-0.01, 0.0, 0.05, 0.10, 0.20, 1.0]
labels = ["0%", "0–5%", "5–10%", "10–20%", "20%+"]
df_clean.loc[:, "discount_bucket"] = pd.cut(df_clean["Discount"], bins=bins, labels=labels)

elasticity_discount = (
    df_clean.groupby("discount_bucket")
    .agg(
        revenue=("Total", "sum"),
        orders=("Id", "count"),
        avg_ticket=("Total", "mean"),
    )
    .reset_index()
)

print(kpis_by_service)
print(kpis_by_payment)
print(kpis_by_region)
print(kpis_by_category.head())
print(elasticity_discount)

## 9. Inferência estatística

In [ ]:
def ic_media(series, alpha=0.05):
    x = series.dropna().values
    n = len(x)
    media = np.mean(x)
    s = np.std(x, ddof=1)
    se = s / np.sqrt(n)
    t_crit = stats.t.ppf(1 - alpha / 2, df=n - 1)
    li = media - t_crit * se
    ls = media + t_crit * se
    return media, se, li, ls


def ic_proporcao(x, alpha=0.05):
    x = np.asarray(x).astype(int)
    n = len(x)
    p_hat = x.mean()
    se = np.sqrt(p_hat * (1 - p_hat) / n)
    z = stats.norm.ppf(1 - alpha / 2)
    li = p_hat - z * se
    ls = p_hat + z * se
    return p_hat, se, li, ls


def checar_normalidade(series):
    from scipy.stats import shapiro

    x = series.dropna()
    if len(x) > 5000:
        x = x.sample(5000, random_state=42)
    stat, p_value = shapiro(x)
    print(f"Shapiro-Wilk p-valor: {p_value:.4f} (H0: normalidade)")
    return stat, p_value


def checar_independencia(series):
    x = series.dropna()
    if len(x) < 3:
        print("Série muito curta para autocorrelação")
        return np.nan
    autocorr = x.autocorr(lag=1)
    print(f"Autocorrelação lag-1: {autocorr:.4f}")
    return autocorr

## 10. Intervalos de confiança e testes

In [ ]:
print("\nIC Ticket Médio:", ic_media(df_clean["Total"]))
checar_normalidade(df_clean["Total"])
checar_independencia(df_clean["Total"])

print("\nIC Atraso Médio:", ic_media(df_clean["delivery_delay_days"]))
checar_normalidade(df_clean["delivery_delay_days"])
checar_independencia(df_clean["delivery_delay_days"])

print("\nIC Atraso (%) :", ic_proporcao(df_clean["is_late"]))
print("\nIC Cancelamentos (%) :", ic_proporcao((df_clean["Purchase_Status"] == "Cancelado").astype(int)))

## 11. Exportações e datasets para BI

In [4]:
kpis_by_service.to_csv("kpis/kpis_by_service.csv", index=False)
kpis_by_payment.to_csv("kpis/kpis_by_payment.csv", index=False)
kpis_by_region.to_csv("kpis/kpis_by_region.csv", index=False)
kpis_by_category.to_csv("kpis/kpis_by_category.csv", index=False)
elasticity_discount.to_csv("kpis/elasticity_discount.csv", index=False)
seasonality.to_csv("kpis/seasonality_month_region.csv", index=False)
df_clean.to_csv("kpis/fact_analytic_clean.csv", index=False)

print("Arquivos exportados para /kpis e figuras para /images.")

NameError: name 'kpis_by_service' is not defined